# Predicting Flight Delays For Domestic Airlines

### Authors: Ethan Bleier, Elijah Kramer, Roberto Palacios

The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS) maintains performance records of domestic flights. These statistics include several interesting variables: dates, taxi times, delays, origins, destinations, departure, and arrival times. 

Using data analysis and various machine learning algorithms, this notebook plans to predict whether or not a flight will experience a delay. Specifically, we are interested in which predictors will play the biggest role in causing flight delays.

Our main dataset is taken from [Kaggle](https://www.kaggle.com/datasets/giovamata/airlinedelaycauses/data) and represents data from 2008. However, we expect to find that many of these same patterns found in this dataset to persist today.

In addition to this, we have [airport data]("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat") and [airline data]("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat") taken from [OpenFlights.org](https://openflights.org/) to help fill in gaps in data and previous knowledge.

# TODO:

Someone Look at TaxiIn and TaxiOut
Feel free to add more

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# scipy
from scipy.stats import zscore

### Reading In Data

In [ ]:
path = kagglehub.dataset_download("giovamata/airlinedelaycauses")
df = pd.read_csv(path + '/DelayedFlights.csv')

port_df = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat")
plane_df = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat")

### Quick plotting settings

In [ ]:
sns.set_theme(context='notebook', style='whitegrid')
plt.rcParams['figure.figsize'] = 6, 4

### Data Preprocessing

The airplane and airport data frames need considerable preprocessing before, they can be of us to use. Likewise, our main data frame could use some cleaning before we perform any analysis.

In [ ]:
df.drop(["Year", "Unnamed: 0", "FlightNum","TailNum", "Cancelled"], axis = 1, inplace = True)
df = df.dropna(subset=['DepDelay', 'ArrDelay'])
df["DepDelayLog"] = np.log10(df["DepDelay"])
df["ArrDelayLog"] = np.log10(df["ArrDelay"])

# Both .dat files need to be formatted properly and new column names are needed
top_row = port_df.columns
port_df.columns = ["AirportId", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST", "TZ", "Type", "Source"]
port_df.loc[-1] = top_row
port_df.index = port_df.index + 1
port_df = port_df.sort_index()

top_row = plane_df.columns
plane_df.columns = ["AirlineId", "ID", "Name", "IATA", "ICAO", "Callsign", "Country", "Active"]
plane_df.loc[-1] = top_row
plane_df.index = plane_df.index + 1
plane_df = plane_df.sort_index()

# plane_df and port_df have columns which we do not need or are of no use so we drop them
port_df = port_df[port_df["Country"] == "United States"]
port_df = port_df[port_df["IATA"] != "\\N"]
port_df.drop(["Source", "ICAO"], axis = 1, inplace = True)

plane_df = plane_df[plane_df["Country"] == "United States"]
plane_df = plane_df.dropna(subset = ["IATA"])
plane_df.drop(["Name", "ICAO"], axis = 1, inplace = True)

In [ ]:
df.info()

In [ ]:
contin_vars = ["DepDelay","ArrDelay","TaxiIn", "TaxiOut", "Distance"]
sns.heatmap(df[contin_vars].corr(), cmap = "PRGn", vmin = -1, vmax = 1);

In [ ]:
plane_df.info()

In [ ]:
port_df.info()

### Initial Data Exploration

While several factors may influence delays, we'd like to examine the constants in this situation rather than rely on chance incidents of weather or unexpected events. As such we'll first be looking at delays in relation to carriers.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (14,6))

median_carrier_delay = df.groupby("UniqueCarrier")[["ArrDelay", "DepDelay"]].median().sort_values(by ="ArrDelay", ascending = False)
mean_carrier_delay = df.groupby("UniqueCarrier")[["ArrDelay", "DepDelay"]].mean().sort_values(by ="ArrDelay", ascending = False)

median_carrier_delay.plot.barh(ax = ax1, title = "Median Delay By Carrier", xlabel = "Median Delay in Minutes");
mean_carrier_delay.plot.barh(ax = ax2, title = "Mean Delay By Carrier", xlabel = "Mean Delay in Minutes");

ax1.set_xlim(0,60)
ax2.set_xlim(0,60)

plt.tight_layout()
plt.show()

### Delays At Airports

This initial analysis suggests that delays are closely tied to carriers. Our dataset also contains a column that relates specifically to carrier delays, which could be used to elucidate some of these figures. However, over a third of these values are null, making further evaluation much more difficult. Instead, we'll explore the relationship between airports and delays.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (14,6))

dest_by_median_delay = df.groupby("Dest")[["ArrDelay","DepDelay"]].median().sort_values(by = "DepDelay", ascending = False).head(15)
dest_by_mean_delay = df.groupby("Dest")[["ArrDelay","DepDelay"]].mean().sort_values(by = "DepDelay", ascending = False).head(15)

dest_by_median_delay.plot.barh(ax = ax1, title = "Top 10 Airports by Median Delay", xlabel = "Median Delay in Minutes");
dest_by_mean_delay.plot.barh(ax = ax2, title = "Top 10 Airports by Mean Delay", xlabel = "Mean Delay in Minutes");

ax1.set_xlim(0, 80)
ax2.set_xlim(0, 80)

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (14,6))

top_ten_airports = df[df["Dest"].isin((df["Dest"].value_counts().head(10)).index)]
top_ten_airports.groupby("Dest")[["ArrDelay","DepDelay"]].median().sort_values(by = "DepDelay", ascending  = False).plot.barh(ax = ax1);

bottom_ten_airports = df[df["Dest"].isin((df["Dest"].value_counts().sort_values().head(10)).index)]
bottom_ten_airports.groupby("Dest")[["ArrDelay","DepDelay"]].median().sort_values(by = "DepDelay", ascending  = False).plot.barh(ax = ax2);

ax1.set_title("Delays At Top 10 Most Visited Airports")
ax1.set_xlabel("Median Delay in Minutes")

ax2.set_title("Delays At Top 10 Least Visited Airports")
ax2.set_xlabel("Median Delay in Minutes")

ax1.set_xlim(0, 60)
ax2.set_xlim(0, 60)

plt.show()

One might expect that busier airports are more likely to suffer flight delays but the data suggests otherwise. Smaller, regional airports appear to suffer equal or higher rates of delays when compared to high-volume airports. The main difference here lies in the consistency of the airports as the 10 most visited airports have similar rates of delay or are at least equal in their rate of arrival delay and departure delay. Low-traffic airports on the other hand vary from one another and even in their arrival and departure delays. 

From this information, we surmise that traffic is not a significant factor causing flight delays. As such we should look at other key variables such as the distance of each flight and the time of year the flight takes place.

In [ ]:
scatter_sample = df[df["CRSArrTime"] < df["ArrTime"]].sample(n = 1000, random_state = 42)

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (14,6))

ax1.scatter(scatter_sample["ArrDelayLog"],scatter_sample["Distance"], alpha = 0.5)


ax1.set_ylim(0, 3000)
ax1.set_title("Scatterplot of Arrival Delay By Distance")
ax1.set_xlabel("Delay")
ax1.set_ylabel("Distance")

ax2.scatter(scatter_sample["DepDelayLog"],scatter_sample["Distance"], alpha = 0.5)

ax2.set_ylim(0, 3000)
ax2.set_title("Scatterplot of Departure Delay By Distance")
ax2.set_xlabel("Delay")
ax2.set_ylabel("Distance")

plt.show()

In [ ]:
scatter_sample = df[df["CRSArrTime"] < df["ArrTime"]].sample(n = 1000, random_state = 42)

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (14,6))
ax1.scatter(scatter_sample["ArrDelay"],scatter_sample["Distance"], alpha = 0.5)

ax1.set_xlim(-5, 300)
ax1.set_ylim(0, 3000)
ax1.set_title("Scatterplot of Arrival Delay By Distance")
ax1.set_xlabel("Delay")
ax1.set_ylabel("Distance")

ax2.scatter(scatter_sample["ArrDelay"],scatter_sample["Distance"], alpha = 0.5)
ax2.set_xlim(0, 50)
ax2.set_ylim(0, 3000)
ax2.set_title("Scatterplot of Arrival Delay By Distance(Zoomed)")
ax2.set_xlabel("Delay")
ax2.set_ylabel("Distance")

plt.show()

Unlike departure delays, some flights show negative values for arrival delays, meaning they arrived at their destinations earlier than expected. As such, we started by isolating flights that failed to arrive on time and then took a sample of those instances. Then, we plotted the arrival delay of these flights against their distance traveled.

The results show that delay times are evenly distributed regardless of the distance traveled, with outliers being the natural result of unforeseen incidents occurring occasionally.

Next, we'll assess how much of an impact the time of year has on flight delays

In [ ]:
months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]

med_month_delay = df.groupby("Month")["ArrDelay"].median()
mean_month_delay = df.groupby("Month")["ArrDelay"].mean()

# applies months labels after grouping
med_month_delay.index = med_month_delay.index.map(lambda x:months[x-1])
mean_month_delay.index = mean_month_delay.index.map(lambda x:months[x-1])

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (14,6))

med_month_delay.sort_values().plot.barh(ax = ax1, title = "Median Arrival Delay By Month", xlabel = "Arrival Delay in Minutes")
mean_month_delay.sort_values().plot.barh(ax = ax2, title = "Mean Arrival Delay By Month", xlabel = "Arrival Delay in Minutes")

ax1.set_xlim(0, 50)
ax2.set_xlim(0, 50)
# applies month labels after grouping. Can't be used with sorting or requires change of label order
# plt.yticks(ticks=range(12), labels = months)
plt.show()

When looking at the arrival delay by month, we notice a trend in the winter and summer months. These months often see much heavier flight traffic than normal due to people taking vacations or visiting family. As a result, arrival delays tend to increase in those months and stay relatively low for the rest of the year.

In [ ]:
# this plot kinda cheeks but it shows all data points
df.boxplot(column = "ArrDelayLog" , by = "Month")
plt.suptitle("")
plt.title("Boxplot of Arrival Delays by Month")
plt.show()

In [ ]:
# this one's pretty...pretty useless
sns.kdeplot(x = "ArrDelayLog", hue = "Month", data = df, common_norm = False)
plt.legend(labels = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"])
plt.show()

In [ ]:
# lets predict if a flight will arrive late using carrier, destination, and date info
(df["ArrDelay"] <= 0).sum()

In [ ]:
df.shape

In [ ]:
# Checking for outliers
plt.boxplot([df['DepDelay'], df['ArrDelay']])
plt.xticks([1, 2], ['DepDelay', 'ArrDelay'])
plt.show()

z_scores = zscore(df[['DepDelay', 'ArrDelay']])
outliers = (abs(z_scores) > 3).any(axis=1)
print("Number of outliers:", outliers.sum())

This plot shows us that there are too many outliers for the mean of the data to provide us with anything useful.

In [ ]:
df_num = df[['ArrDelay', 'DepDelay']]
numeric_vars = df_num.columns.values

zscore1 = lambda s: zscore(s, nan_policy='omit')

df_scaled = df_num.apply(zscore1)

In [ ]:
df_scaled.apply(['min', 'max']).round(2).transpose()

The calculation of the min and max in this instance shows us that some delay values are negative and may indicate either flights that arrived early, or some sort of error.

In [ ]:
sns.pairplot(df_num);

In [ ]:
q01, q99 = df[['DepDelay', 'ArrDelay']].quantile([0.01, 0.99]).values

df['DepDelay'] = df['DepDelay'].clip(q01[0], q99[0])
df['ArrDelay'] = df['ArrDelay'].clip(q01[1], q99[1])

X = StandardScaler().fit_transform(df[['DepDelay', 'ArrDelay']])
kmeans = KMeans(n_clusters=3, random_state=0)
clusters = kmeans.fit_predict(X)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.scatter(df['DepDelay'], df['ArrDelay'], alpha=0.5)
ax1.set_title('Raw Data')
ax1.set_xlabel('Departure Delay')
ax1.set_ylabel('Arrival Delay')

ax2.scatter(df['DepDelay'], df['ArrDelay'], alpha=0.5, c=clusters)
ax2.set_title('Clipped Data with Clusters')
ax2.set_xlabel('Departure Delay')
ax2.set_ylabel('Arrival Delay')

plt.tight_layout()
plt.show()

In [ ]:
z_scores = zscore(df[['DepDelay', 'ArrDelay']])
outliers_removed = df[abs(z_scores).max(axis=1) <= 3]

### Machine Learning

In [ ]:
X = df[['DepDelay', 'ArrDelay']]

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

kmeans = KMeans(n_clusters=3,random_state=0)
cluster_nums = kmeans.fit_predict(X)

Sanity check: Are the clusters too skewed?

In [ ]:
cluster_nums[:100]

print("Cluster 1:", np.bincount(cluster_nums)[0])
print("Cluster 2:", np.bincount(cluster_nums)[1]) 
print("Cluster 3:", np.bincount(cluster_nums)[2])

These clusters seem to expose an acceptable level of skewedness.

In [ ]:
pd.DataFrame(kmeans.cluster_centers_, columns=['RedMeat', 'Fr&Veg'])

I also want to see the clusters

In [ ]:
centers = pd.DataFrame(kmeans.cluster_centers_,
columns=['ArrDelay', 'DepDelay'])

centers.plot.barh()
plt.title('Cluster centers')
plt.xlabel('value'); plt.ylabel('cluster number')
plt.show()

Now in 2D

In [ ]:
cluster_labels = {
    0: 'Low Delay',
    1: 'Medium Delay',
    2: 'High Delay'
}

labels = [cluster_labels[c] for c in clusters]

plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=clusters, label=labels)
plt.legend(['Low Delay', 'Medium Delay', 'High Delay'])
plt.tight_layout()

In [ ]:
X2 = PCA(n_components=2).fit_transform(X)
ax = sns.scatterplot(x=X2[:,0], y=X2[:,1], hue=cluster_nums,
palette='Set1', s=20, legend=True)
ax.set_title('Flight delays')
ax.set_xlabel('1st principal component')
ax.set_ylabel('2nd principal component');
plt.legend(['Low Delay', 'High Delay', 'Medium Delay']);

I don't think these predictor variable are working out. Im going to try distance and airport.

In [ ]:
df_num = df[['Distance', 'DepDelay']]
numeric_vars = df_num.columns.values

df_scaled = df_num.apply(lambda s: zscore(s, nan_policy='omit'))

In [ ]:
airport_stats = df.groupby('Origin').agg({
   'Distance': 'mean',
   'CarrierDelay': 'mean'
}).dropna()

X_scaled = scaler.fit_transform(airport_stats)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0)
airport_stats['Cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
# Blue is decent, red is bad and blue is good airport
colors = ['#1f77b4', '#d62728', '#2ca02c']  # RGB

plt.scatter(X_scaled[:, 0], X_scaled[:, 1],
           c=[colors[i] for i in airport_stats['Cluster']],
           s=30, alpha=0.3)

distances = np.sqrt(X_scaled[:, 0]**2 + X_scaled[:, 1]**2)
outlier_threshold = np.percentile(distances, 90)  # Top 10% as outliers
outliers = distances > outlier_threshold

for i, txt in enumerate(airport_stats.index):
    if outliers[i]:
        plt.annotate(txt, (X_scaled[i, 0], X_scaled[i, 1]), 
                    fontsize=8, xytext=(5, 5),
                    textcoords='offset points')

plt.xlabel('Average Flight Distance (scaled)')
plt.ylabel('Average Airport Delay (scaled)')
plt.title('Airport Clusters by Distance and Delay')
plt.show();

Next up, let's try using a regression algorithm to predict flight delay. First off, we perform a test/train split on our data. Let's use a test size of 0.15 to preserve a good amount of data for training

In [ ]:
X = df['DepDelay']
y = df['Distance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

print(f'Using {X_train.shape[0]} samples for training and {X_test.shape[0]} for testing')